In [335]:
import numpy as np
import pandas as pd
import scholarly
import sqlite3
import itertools

from sqlalchemy import create_engine

In [337]:
keysearch = input("Enter the keyword to search Google Scholar: ")

Enter the keyword to search Google Scholar: cerebellum


In [250]:
conn = sqlite3.connect('./kolDB.db')

In [341]:
engine = create_engine("sqlite:///kolDB.db", echo=False)

In [251]:
c = conn.cursor()

In [181]:
# Creating the "edgelist" table

c.execute(
    '''
    CREATE TABLE edgelist (
    author1 text, 
    author2 text, 
    title text, 
    url text, 
    journal text)
    '''
)

In [43]:
# Creating the "metrics" table

c.execute(
    '''
    CREATE TABLE metrics (
    ID text, 
    author text, 
    cited integer, 
    cited5y integer, 
    h_index integer, 
    h_index5y integer, 
    i10index integer, 
    i10index5y integer
    )
    '''
)


In [23]:
c.execute(
    '''
    CREATE TABLE background (
    ID text,
    author text,
    interests text
    )
    '''
)

In [224]:
c.execute("INSERT INTO employees VALUES ('Ryan', 'Willett', 96000)")

In [226]:
c.execute("SELECT * FROM employees WHERE last = 'Willett'")

In [227]:
c.fetchall()

[('Ryan', 'Willett', 96000)]

In [261]:
conn.commit()

In [17]:
conn.close()

In [344]:
cb_search = scholarly.search_keyword(keysearch)

In [18]:
# How many elements is in the generator? 
sum(1 for i in cb_search)

828

In [12]:
# for i in list(range(6)):
#     print(next(cb_search))

StopIteration: 

In [22]:
# cb_author = next(cb_search).fill()

In [23]:
print(cb_author)

{'_filled': True,
 'affiliation': 'Queensland University of Technology',
 'citedby': 59993,
 'citedby5y': 45989,
 'cites_per_year': {2003: 165,
                    2004: 235,
                    2005: 335,
                    2006: 297,
                    2007: 450,
                    2008: 522,
                    2009: 725,
                    2010: 803,
                    2011: 1056,
                    2012: 1238,
                    2013: 2660,
                    2014: 4420,
                    2015: 5798,
                    2016: 6615,
                    2017: 8035,
                    2018: 10680,
                    2019: 14216,
                    2020: 379},
 'coauthors': [<scholarly.scholarly.Author object at 0x1057e0cf8>,
 'email': '@qut.edu.au',
 'hindex': 88,
 'hindex5y': 66,
 'i10index': 327,
 'i10index5y': 278,
 'id': 'gJUlfqcAAAAJ',
 'interests': ['air quality', 'aerosol science'],
 'name': 'Lidia Morawska',
 'publications': [<scholarly.scholarly.Publication obje

In [377]:
def var_extract(auth):
    '''
    Take a Scholarly author object and extracts informatino about the author and their publications. 
    '''
    
    # Collecting simple variables
    record_name = auth.name
    record_id = auth.id
    record_interests = auth.interests
    affiliation = auth.affiliation
    cited = auth.citedby
    cited5y = auth.citedby5y
    h_index = auth.hindex
    h_index5y = auth.hindex5y
    i10index = auth.i10index
    i10index5y = auth.i10index5y
    
    print("\nExtracting information for author: " + record_name + "\n")
    
    # Assembling SQL command to write data to the "metrics" database
    sql_s = "INSERT INTO metrics VALUES(\"" + str(record_id) + "\" , \"" + str(record_name) + "\" , " + str(cited)  + "," + str(cited5y) + "," + str(h_index) + "," + str(h_index5y) + "," + str(i10index) + "," + str(i10index5y) + ")"
#     print(sql_s)
    c.execute(sql_s)
    conn.commit()
    
    # Assigning the co-authors listed by the user on Google Scholar
#     coauth_manual = [auth.coauthors[i].name for i in range(len(auth.coauthors))]

    # Extracts relevant information from each publication in an author's record
    cl = [pub_extract(auth, i) for i in range(3)] #len(auth.publications))]
    
    # Unpack data from publication extraction into dataframe and upload to database
    [pub_unpacker(l, record_name) for l in cl]
    
    for l in cl: 
        sql_df = pub_unpacker(l, record_name)
        
    
#     return pd.concat(testOut)
    
def pub_unpacker(d, name): 
    
    coauths, pub_title, pub_url, pub_journal = d
    
#     cl_flat = list(itertools.chain.from_iterable(coauths)) # Flattening list of lists
    
    coauths.append(name)
    
    print(coauths)
    print(len(coauths))
    
    if len(coauths) >1: 
    
        pubEL = edgelister(coauths)

        pubEL["title"] = pub_title
        pubEL["url"] = pub_url
        pubEL["journal"] = pub_journal

        
        for r in range(pubEL.shape[0]):
#             ul = (pubEL.author1[r], pubEL.author2[r], pub_title, pub_url, pub_journal)
            c.executemany("INSERT INTO edgelist VALUES (?,?,?,?,?)", (pubEL.author1[r], pubEL.author2[r], pub_title, pub_url, pub_journal))
            print(ul)
            conn.commit()
            # Clean up the dataframe
        # Removing connection to self (coauthor = author)
    #     pubEL = pubEL.loc[pubEL.author1 != pubEL.author2]

        # Removing duplicate connections/edges for each author
#         pubEL.drop_duplicates(inplace=True)

        # Enter the dataframe into a database
#         pubEL.to_sql('edgelist', con = conn, index=False, if_exists='append')
        
    #     return pubEL
    
    else: 
        print("No coauthors found")
        
def pub_unpacker_orig(d, name): 
    
    coauths, pub_title, pub_url, pub_journal = d
    
#     cl_flat = list(itertools.chain.from_iterable(coauths)) # Flattening list of lists
    
    coauths.append(name)
    
    print(coauths)
    print(len(coauths))
    
    if len(coauths) >1: 
    
        pubEL = edgelister(coauths)

        pubEL["title"] = pub_title
        pubEL["url"] = pub_url
        pubEL["journal"] = pub_journal

            # Clean up the dataframe
        # Removing connection to self (coauthor = author)
    #     pubEL = pubEL.loc[pubEL.author1 != pubEL.author2]

        # Removing duplicate connections/edges for each author
        pubEL.drop_duplicates(inplace=True)

        # Enter the dataframe into a database
        pubEL.to_sql('edgelist', con = conn, index=False, if_exists='append')
        
    #     return pubEL
    
    else: 
        print("No coauthors found")        

def pub_unpacker_alchemy(d, name): 
    
    coauths, pub_title, pub_url, pub_journal = d
    
#     cl_flat = list(itertools.chain.from_iterable(coauths)) # Flattening list of lists
    
    coauths.append(name)
    
    if len(coauths) >1: 
    
        pubEL = edgelister(coauths)

        pubEL["title"] = pub_title
        pubEL["url"] = pub_url
        pubEL["journal"] = pub_journal

            # Clean up the dataframe
        # Removing connection to self (coauthor = author)
    #     pubEL = pubEL.loc[pubEL.author1 != pubEL.author2]

        # Removing duplicate connections/edges for each author
        pubEL.drop_duplicates(inplace=True)

        # Enter the dataframe into a database
        pubEL.to_sql('edgelist', con = engine, index=False, if_exists='append')

        
        
    #     return pubEL
    
    else: 
        print("No coauthors found")        
        
def edgelister(simple_list): 
    
    listSet1 = []
    for i in range(len(simple_list)-1):
        test = [simple_list[i]] * (len(simple_list) -1 -i)
        listSet1.append(test)
        
    listSet2 = []
    for i in range(1, len(simple_list)):
#         print(simple_list[i:])
        test2 = simple_list[i:]
        listSet2.append(test2)
    
    listSet1_expanded = list(itertools.chain.from_iterable(listSet1))
    listSet2_expanded = list(itertools.chain.from_iterable(listSet2))
    
    el_df = pd.DataFrame({'author1': listSet1_expanded, 'author2': listSet2_expanded})
    
    return el_df

def pub_extract(auth, i): 
    
    '''
    Expands the publications field of the scholarly author object and extracts the coauthor names and manuscript 
    title from each publication. Returns a tuple of coauthor (list), pub title (string), pub url (string) and the
    journal name (string).
    '''
    
    try: 
        pubStub = auth.publications[i].fill()
        coauths = pubStub.bib["author"].split(" and ") # Make list from coauthor string
        coauths = [ca.strip() for ca in coauths] # Remove leading and trailing whitespace
        
        print("Authors for publication " + str(i))
        print("=" * 60)
        [print(ca) for ca in coauths] # Print the list of names extracted
        print("\n")
        
        pub_title = pubStub.bib["title"]
        pub_url = pubStub.bib["url"]
        pub_journal = pubStub.bib["journal"]
        
        return (coauths, pub_title, pub_url, pub_journal)
    except: 
        return([], "", "", "")

def generator_db(gen):
    
    '''
    Opens up a connection to a sqlite database, iterates through the scholarly generator and extracts the information
    for each author.
    '''
    
    conn = sqlite3.connect('./kolDB.db')
#     c = conn.cursor()
    
    for i in list(range(6)):
        var_extract(next(gen).fill())
    
#     conn.commit() # Commits the SQL queries specified by functions to the sqlite datebase.
    conn.close()

In [378]:
generator_db(cb_search)


Extracting information for author: douglas r wylie

Authors for publication 1
CI De Zeeuw
DR Wylie
PL DiGiorgi
JI Simpson


Authors for publication 2
CI De Zeeuw
DR Wylie
JS Stahl
JI Simpson


['douglas r wylie']
1
No coauthors found
['CI De Zeeuw', 'DR Wylie', 'PL DiGiorgi', 'JI Simpson', 'douglas r wylie']
5


ProgrammingError: Incorrect number of bindings supplied. The current statement uses 5, and there are 11 supplied.

['Ilana Ackerman']

In [356]:
listTest = ["Egidio D'Angelo"
'Thierry Nieus',
'Arianna Maffei',
'Simona Armano',
'Paola Rossi',
'Vanni Taglietti',
'Andrea Fontana',
'Giovanni Naldi']

In [362]:
df_edge = edgelister(listTest)
df_edge

,author1,author2
0,Egidio D'AngeloThierry Nieus,Arianna Maffei
1,Egidio D'AngeloThierry Nieus,Simona Armano
2,Egidio D'AngeloThierry Nieus,Paola Rossi
3,Egidio D'AngeloThierry Nieus,Vanni Taglietti
4,Egidio D'AngeloThierry Nieus,Andrea Fontana
5,Egidio D'AngeloThierry Nieus,Giovanni Naldi
6,Arianna Maffei,Simona Armano
7,Arianna Maffei,Paola Rossi
8,Arianna Maffei,Vanni Taglietti
9,Arianna Maffei,Andrea Fontana


In [361]:
df_edge.T.values[0]

array(["Egidio D'AngeloThierry Nieus", "Egidio D'AngeloThierry Nieus",
       "Egidio D'AngeloThierry Nieus", "Egidio D'AngeloThierry Nieus",
       "Egidio D'AngeloThierry Nieus", "Egidio D'AngeloThierry Nieus",
       'Arianna Maffei', 'Arianna Maffei', 'Arianna Maffei',
       'Arianna Maffei', 'Arianna Maffei', 'Simona Armano',
       'Simona Armano', 'Simona Armano', 'Simona Armano', 'Paola Rossi',
       'Paola Rossi', 'Paola Rossi', 'Vanni Taglietti', 'Vanni Taglietti',
       'Andrea Fontana'], dtype=object)

In [365]:
pub_title = "Awesome Title"
pub_url = "http://www.awesome-journal.com"
pub_journal = "Awesome Journal"

for r in range(df_edge.shape[0]):
    ul = (df_edge.author1[r], df_edge.author2[r], pub_title, pub_url, pub_journal)
    conn.executemany("INSERT INTO edgelist VALUES(?,?,?,?,?)")
    print(ul)

("Egidio D'AngeloThierry Nieus", 'Arianna Maffei', 'Awesome Title', 'http://www.awesome-journal.com', 'Awesome Journal')
("Egidio D'AngeloThierry Nieus", 'Simona Armano', 'Awesome Title', 'http://www.awesome-journal.com', 'Awesome Journal')
("Egidio D'AngeloThierry Nieus", 'Paola Rossi', 'Awesome Title', 'http://www.awesome-journal.com', 'Awesome Journal')
("Egidio D'AngeloThierry Nieus", 'Vanni Taglietti', 'Awesome Title', 'http://www.awesome-journal.com', 'Awesome Journal')
("Egidio D'AngeloThierry Nieus", 'Andrea Fontana', 'Awesome Title', 'http://www.awesome-journal.com', 'Awesome Journal')
("Egidio D'AngeloThierry Nieus", 'Giovanni Naldi', 'Awesome Title', 'http://www.awesome-journal.com', 'Awesome Journal')
('Arianna Maffei', 'Simona Armano', 'Awesome Title', 'http://www.awesome-journal.com', 'Awesome Journal')
('Arianna Maffei', 'Paola Rossi', 'Awesome Title', 'http://www.awesome-journal.com', 'Awesome Journal')
('Arianna Maffei', 'Vanni Taglietti', 'Awesome Title', 'http://www.